In [1]:
from IPython.core.display import display, HTML
text='''<h2>Articles of Lu\'Xun Search Program</h2>
<p>By <a href='https://github.com/ginchung'>Ginchung@Github</a><br/>On May 8th, 2019@Hefei<br/></p>
<h3>转载请标明出处：本Notebook使用MIT许可(MIT License)</h3>
<h3>Dependencies：</h3><p>Python3 Platform<br>Whoosh&nbsp;(检索)<br>Jieba&nbsp;(用于分词)<br>Ac-heron Depo: luxun&nbsp;
<a href="https://github.com/Ac-heron/luxun">鲁迅文集相关资料</a></p>
<h3>如果需要直接使用，请直接运行最后一个Cell<br>Welcome to STAR ME!</h3>'''
display(HTML(text))

In [2]:
#coding='utf-8'
# 文章保存的根目录
ARTICLE_LOCATION = 'article'

import os
articles=[]
dirs = os.listdir(ARTICLE_LOCATION)
for ARTICLE_DIR in dirs:
    # 在文章的根目录下寻找发表的文集名目录
    NAME_ARR=os.listdir('%s/%s'%(ARTICLE_LOCATION,ARTICLE_DIR))
    for name in NAME_ARR:
        # 文件名包含“简介”或者“目录”二字，不是鲁迅作
        if '简介' not in name and '目录' not in name:
            articles.append('%s/%s/%s'%(ARTICLE_LOCATION,ARTICLE_DIR,name))
print('文章数量有：%d篇'%len(articles))

文章数量有：306篇


In [3]:
# 代码快速实现参考：https://cloud.tencent.com/developer/article/1374967

import os
from whoosh.index import create_in
from whoosh.fields import *
from jieba.analyse import ChineseAnalyzer
import json

# 使用结巴中文分词
analyzer = ChineseAnalyzer()

# 创建schema, stored为True表示能够被检索
schema = Schema(title=TEXT(stored=True, analyzer=analyzer), path=ID(stored=False),content=TEXT(stored=True, analyzer=analyzer))

# 存储schema信息至'indexdir'目录下
ix_path = 'indexdir/'
ix_name = 'luxun_index_name'

if not os.path.exists(ix_path):
    os.mkdir(ix_path)
ix = create_in(ix_path, schema,indexname=ix_name)
ix.close()

In [4]:
from whoosh.filedb.filestore import FileStorage
import re

# Replace pattern
rpp=re.compile('[\u3000※·\n\r〔〕 ]')

storage = FileStorage(ix_path)  #idx_path 为索引路径
ix = storage.open_index(indexname=ix_name)
# 按照schema定义信息，增加需要建立索引的文档
# 注意：字符串格式需要为unicode格式
with ix.writer() as w:
    for article in articles:
        with open(article,'r',encoding='utf-8') as f:
            arr=f.read()
        subs=re.split(rpp,arr)
        art=''.join(subs)
        length=len(art)
        ind=art[::-1].find('〔１〕')
        if ind>0:
            art=art[:length-ind-1]
            art=re.split('[\d]',art)
            art=''.join(art[1:])
        name_arr=article.split('/')
        w.add_document(title='集名：%s 篇名：%s'%(name_arr[1],name_arr[-1][:-3]),content=art)
        print("已引入文章：%s"%article)
        
ix.close()
print("索引已建立好")

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Lithium\AppData\Local\Temp\jieba.cache
Loading model cost 0.772 seconds.
Prefix dict has been built succesfully.


已引入文章：article/三闲集/“皇汉医学”.md
已引入文章：article/三闲集/“醉眼”中的朦胧.md
已引入文章：article/三闲集/“革命军马前卒”和“落伍者”.md
已引入文章：article/三闲集/《吾国征俄战史之一页》.md
已引入文章：article/三闲集/《小彼得》译本序.md
已引入文章：article/三闲集/《近代世界短篇小说集》小引.md
已引入文章：article/三闲集/书籍和财色.md
已引入文章：article/三闲集/匪笔三篇.md
已引入文章：article/三闲集/叶永蓁作《小小十年》小引.md
已引入文章：article/三闲集/吊与贺.md
已引入文章：article/三闲集/在钟楼上（夜记之二）.md
已引入文章：article/三闲集/太平歌诀.md
已引入文章：article/三闲集/头.md
已引入文章：article/三闲集/序言.md
已引入文章：article/三闲集/怎么写（夜记之一）.md
已引入文章：article/三闲集/我和《语丝》的始终.md
已引入文章：article/三闲集/我的态度气量和年纪.md
已引入文章：article/三闲集/扁.md
已引入文章：article/三闲集/文坛的掌故（并徐匀来信）.md
已引入文章：article/三闲集/文学的阶级性（并恺良来信）.md
已引入文章：article/三闲集/文艺与革命（并冬芬来信）.md
已引入文章：article/三闲集/新月社批评家的任务.md
已引入文章：article/三闲集/无声的中国.md
已引入文章：article/三闲集/某笔两篇.md
已引入文章：article/三闲集/柔石作《二月》小引.md
已引入文章：article/三闲集/流氓的变迁.md
已引入文章：article/三闲集/现今的新文学的概观.md
已引入文章：article/三闲集/路.md
已引入文章：article/三闲集/辞顾颉刚教授令“候审”（并来信）.md
已引入文章：article/三闲集/述香港恭祝圣诞.md
已引入文章：article/三闲集/通信（并Ｙ来信）.md
已引入文章：article/三闲集/铲共大观.md
已引入文章：article/三闲集/革命咖啡店.md
已引入文章：article/三闲集/鲁迅译著书目.m

已引入文章：article/野草/风筝.md
索引已建立好


In [5]:
from IPython.core.display import display, HTML
from whoosh.filedb.filestore import FileStorage
import json

ix_path = 'indexdir/'
ix_name = 'luxun_index_name'
storage = FileStorage(ix_path)

with storage.open_index(indexname=ix_name).searcher() as searcher:
    # 检索标题中出现'文档'的文档
    results = searcher.find("content",input("请输入你要检索的文字："))
    # 检索出来的第一个结果，数据格式为dict{'title':.., 'content':...}
    for r in results:
        display(HTML('<h3>'+r.get('title')+'</h3>'))
        display(HTML(r.highlights("content")))  
        # 高亮标题中的检索词
        print('相似度评分：%.2f'%r.score)  
        # 准确度分数
        print('文章序号：%d'%r.docnum)
        
        doc = r.fields()
        jsondoc = json.dumps(doc, ensure_ascii=False)
        # display(jsondoc)  
        # 打印出检索出的文档全部内容


请输入你要检索的文字：拿衣服


相似度评分：9.38
文章序号：192


相似度评分：8.47
文章序号：253


相似度评分：8.06
文章序号：286


相似度评分：7.92
文章序号：152


...

相似度评分：7.13
文章序号：180


相似度评分：6.88
文章序号：190


相似度评分：6.75
文章序号：193


相似度评分：6.49
文章序号：206


相似度评分：5.59
文章序号：202
